<a href="https://colab.research.google.com/github/ajay26296thakkar/Fake_News/blob/master/Fake_Lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words=set(stopwords.words("english"))
import numpy as np
import random
import gensim
from gensim.models import Word2Vec
import string
nltk.download('punkt')
from google.colab import drive
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
drive.mount('/content/gdrive')
stop_words = set(stopwords.words('english'))
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC  
svclassifier = SVC(kernel='linear')  
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,classification_report

def load_kagglefakenews():
    #load training data and put into arrays
    df = pd.read_csv('gdrive/My Drive/data.csv', encoding='utf8') 
    train_data = df['Body'].values.tolist() #'text' column contains articles
    train_labels = df['Label'].values.tolist() #'label' column contains labels

    #Randomly shuffle data and labels together
    combo = list(zip(train_data, train_labels))
    #random.shuffle(combo)
    train_data, train_labels = zip(*combo)
    del df #clear up memory
   
    return np.asarray(train_data).tolist(), np.asarray(train_labels).tolist()
def PreProcessing(train_da,train_lab):
  i=0
  for text in train_da:
    tokens = word_tokenize(text)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in tokens if word.isalpha()]
    words = [w for w in words if not w in stop_words]
    stemmed = [porter.stem(word) for word in words]
    train_da[i]=stemmed
    i=i+1
  return train_da,train_lab

train_data, train_labels = load_kagglefakenews()
train_data, train_labels = PreProcessing(train_data, train_labels)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
model=Word2Vec(train_data[100:250],size=5,window=2,sg=1,min_count=0,alpha=0.01)
model.train(train_data[100:250], total_examples=150, epochs=5)
word=model.wv.vocab
t_dict=dict(zip(list(word),model[word]))
X_train, X_test, y_train, y_test = train_test_split(train_data[100:250], train_labels[100:250] , test_size=0.20,shuffle=False)
Features = list(map(lambda y:list(np.sum(list(map(lambda x:list(t_dict[x]),y)),axis=0)),X_train))
Te_Features = list(map(lambda y:list(np.sum(list(map(lambda x:list(t_dict[x]),y)),axis=0)),X_test))
svclassifier.fit(Features, y_train)  
y_pred = svclassifier.predict(Te_Features)  
accuracy=accuracy_score(y_test,y_pred)
print(accuracy)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


0.75
